# Feature Engineering

## Libraries

In [1]:
import numpy as np
import pandas as pd
from cnr_methods import get_simplified_data 

# Feature Engineering Library for Time Series
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table

from sklearn.ensemble import RandomForestRegressor
# Feature Selection Library
from boruta import BorutaPy

## Read Data

For this pipeline, only Training Set will be used.

In [2]:
full_data = get_simplified_data()
full_data = full_data[full_data['Set']=='Train']
y_train = pd.read_csv('Y_train.csv')

As done in the other Notebooks, we will transform the Column 'Time' to Datetime format and set as the index of the dataset.

In [3]:
full_data['Time'] = pd.to_datetime(full_data['Time'],dayfirst=True)
full_data = full_data.set_index('Time')

In [4]:
full_data.head()

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
Time,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440,82.543144,Train
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260,99.990844,Train
2018-05-01 03:00:00,3,WF1,-1.220571,-0.266871,0.997093,-1.415138,286.575,98.367235,Train
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780,94.860604,Train
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460,95.905879,Train


To simplify the work, we will generate features for just one Wind Farm. When doing modelling, the features, as the models, will be generated for all Wind Farms separately.

In [5]:
WF = 'WF1'
data = full_data[full_data['WF']==WF]
y_train = y_train[y_train['ID'].isin(data['ID'])]

## Feature Creation

### Wind Speed Vector

In [6]:
feature_data = data[['ID','WF','U_100m','V_100m','U_10m','V_10m','T','CLCT','Set']]
feature_data['Wind Speed 100m'] = np.sqrt(feature_data['U_100m']**2 + feature_data['V_100m']**2)
feature_data['Wind Direction 100m'] = np.arctan(feature_data['V_100m']/feature_data['U_100m'])
feature_data['Wind Speed 10m'] = np.sqrt(feature_data['U_10m']**2 + feature_data['V_10m']**2)
feature_data['Wind Direction 10m'] = np.arctan(feature_data['V_10m']/feature_data['U_10m'])
feature_data = feature_data.drop(['U_100m','V_100m','U_10m','V_10m'],axis=1)

Changing Reference for Negative Angle:

In [7]:
feature_data[feature_data['Wind Direction 100m'] < 0]['Wind Direction 100m'] = 360 - feature_data[feature_data['Wind Direction 100m'] < 0]['Wind Direction 100m']
feature_data[feature_data['Wind Direction 10m'] < 0]['Wind Direction 10m'] = 360 - feature_data[feature_data['Wind Direction 10m'] < 0]['Wind Direction 10m']

Using Wind Speed and Direction instead of U and V, we will create some variables over the Numerical Variables from the simplified data.

In [8]:
features = ['T', 'CLCT', 'Wind Speed 100m','Wind Direction 100m', 'Wind Speed 10m', 'Wind Direction 10m']

### Time-Relative Variables

Here, we get Values for Last Week and Month for each Numerical Feature.

In [9]:
for column in features:
    feature_data[column + '_last_week'] = feature_data[column].shift(7)
    feature_data[column + '_last_month'] = feature_data[column].shift(30)

Get the Number of Month:

In [10]:
feature_data['Month_Number'] = feature_data.index.month

Month Statistics:

In [11]:
mean = feature_data.groupby('Month_Number').mean()[features]
median = feature_data.groupby('Month_Number').median()[features]
variance = feature_data.groupby('Month_Number').var()[features]

In [12]:
mean.columns = mean.columns + '_Month_Mean'
median.columns = median.columns + '_Month_Median'
variance.columns = variance.columns + '_Month_Variance'

In [13]:
feature_data = feature_data.merge(mean,on='Month_Number',how='left')
feature_data = feature_data.merge(median,on='Month_Number',how='left')
feature_data = feature_data.merge(variance,on='Month_Number',how='left')

In [14]:
feature_data.index = data.index

### Distance from Features

Distance of Position of Max and Min (Already on Tsfresh, check it later):

In [15]:
for column in features:
    feature_data[column + '_Distance_Max'] = feature_data.index - feature_data[column].idxmax()
    feature_data[column + '_Distance_Min'] = feature_data.index - feature_data[column].idxmin()
    feature_data[column + '_Distance_Max'] = feature_data[column + '_Distance_Max'].apply(lambda x : x.days)
    feature_data[column + '_Distance_Min'] = feature_data[column + '_Distance_Min'].apply(lambda x : x.days)

### Rolling Window Variables

### Wavelet Transformations (Check)

In [16]:
feature_data = feature_data.fillna(0)

## Tsfresh

Now we use Tsfresh, a Python Library that automates Feature Engineering for Time Series Data. We generate new features for all the columns on the Simplified Data, as done below.

In [17]:
data = data[['ID','WF','U_100m','V_100m','U_10m','V_10m','T','CLCT','Set']]

In [18]:
tsfresh_data = pd.DataFrame()
for variable in ['U_100m','V_100m','U_10m','V_10m','T','CLCT']: 
    df_shift, y = make_forecasting_frame(data[variable],kind=variable,max_timeshift=20,rolling_direction=1)
    X = extract_features(df_shift.drop('kind',axis=1),y,column_id="id", column_sort="time",show_warnings=False,n_jobs=3)
    X['Feature'] = variable
    tsfresh_data = tsfresh_data.append(X)

Feature Extraction: 100%|██████████| 15/15 [01:49<00:00,  7.27s/it]


Process tsfresh_data to pass column 'Features' to the other columns

In [19]:
tsfresh_data = tsfresh_data.pivot(columns='Feature')

In [20]:
tsfresh_data.columns = tsfresh_data.columns.map('{0[0]}|{0[1]}'.format)

In [21]:
tsfresh_data.head()

,"value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_11__w_10|CLCT","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_11__w_10|T","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_11__w_10|U_100m","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_11__w_10|U_10m","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_11__w_10|V_100m","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_11__w_10|V_10m",value__minimum|CLCT,value__minimum|T,value__minimum|U_100m,value__minimum|U_10m,...,value__symmetry_looking__r_0.4|U_100m,value__symmetry_looking__r_0.4|U_10m,value__symmetry_looking__r_0.4|V_100m,value__symmetry_looking__r_0.4|V_10m,value__large_standard_deviation__r_0.2|CLCT,value__large_standard_deviation__r_0.2|T,value__large_standard_deviation__r_0.2|U_100m,value__large_standard_deviation__r_0.2|U_10m,value__large_standard_deviation__r_0.2|V_100m,value__large_standard_deviation__r_0.2|V_10m
id,,,,,,,,,,,,,,,,,,,,,
2018-05-01 02:00:00,133.29164,970.684788,3.316161,2.270426,-5.559159,-3.878295,82.543144,286.44,-2.2485,1.254603,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2018-05-01 03:00:00,133.29164,970.684788,3.316161,2.270426,-5.559159,-3.878295,82.543144,286.26,-2.4345,1.254603,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2018-05-01 04:00:00,133.29164,970.684788,3.316161,2.270426,-5.559159,-3.878295,82.543144,286.26,-2.4345,0.997093,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2018-05-01 05:00:00,133.29164,970.684788,3.316161,2.270426,-5.559159,-3.878295,82.543144,284.78,-2.4345,0.689598,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2018-05-01 06:00:00,133.29164,970.684788,3.316161,2.270426,-5.559159,-3.878295,82.543144,284.46,-2.4345,0.290994,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [22]:
tsfresh_data = tsfresh_data.fillna(0)

In [23]:
U_100m = []
U_10m = []
V_100m = []
V_10m = []
T = []
CLCT = []


for column in tsfresh_data.columns:
    if column.__contains__('|U_100m'):
        U_100m.append(column)
    elif column.__contains__('|U_10m'):
        U_10m.append(column)
    elif column.__contains__('|V_100m'):
        V_100m.append(column)
    elif column.__contains__('|V_10m'):
        V_10m.append(column)
    elif (column.__contains__('|T')):
        T.append(column)
    elif (column.__contains__('|CLCT')):
        CLCT.append(column)

In [32]:
tsfresh_features_filtered = []
for feature in [U_100m,U_10m,V_100m,V_10m,T,CLCT]:
    relevance_table = calculate_relevance_table(tsfresh_data[feature],y,n_jobs=3)
    tsfresh_features_filtered.append(relevance_table[relevance_table['relevant']==True].sort_values('p_value')['feature'].values[:10]) #Filter for Relevant Features Only and Maybe just the first 10
tsfresh_features_filtered = np.concatenate(tsfresh_features_filtered)

In [33]:
tsfresh_features_filtered

array(['value__c3__lag_1|U_100m', 'value__sum_values|U_100m',
       'value__fft_coefficient__coeff_0__attr_"real"|U_100m',
       'value__mean|U_100m', 'value__c3__lag_2|U_100m',
       'value__quantile__q_0.7|U_100m',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_10__w_20|U_100m',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_11__w_20|U_100m',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_12__w_20|U_100m',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_20|U_100m',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_12__w_20|U_10m',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_11__w_20|U_10m',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_13__w_20|U_10m',
       'value__fft_coefficient__coeff_0__attr_"real"|U_10m',
       'value__sum_values|U_10m',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_14__w_20|U_10m',
       'value__cwt_coefficients__widths_(2, 5, 10,

## Feature Selection

In [35]:
final_features = pd.concat([feature_data,tsfresh_data],axis=1)

In [40]:
final_features = final_features.fillna(0)

Here we do the Feature Selection using Borutapy, a Python Implementation of the Famous R Method. For the method we use a Random Forest Regressor.

In [41]:
rf = RandomForestRegressor(n_estimators=100,n_jobs=3, max_depth=5)

In [42]:
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)

In [43]:
feat_selector.fit(final_features.drop(['ID','WF','Set'],axis=1).values, y_train['Production'].values)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	109
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	109
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	109
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	109
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	109
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	109
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	109
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	21
Tentative: 	32
Rejected: 	56
Iteration: 	9 / 100
Confirmed: 	21
Tentative: 	32
Rejected: 	56
Iteration: 	10 / 100
Confirmed: 	21
Tentative: 	32
Rejected: 	56
Iteration: 	11 / 100
Confirmed: 	21
Tentative: 	32
Rejected: 	56
Iteration: 	12 / 100
Confirmed: 	23
Tentative: 	30
Rejected: 	56
Iteration: 	13 / 100
Confirmed: 	23
Tentative: 	30
Rejected: 	56
Iteration: 	14 / 100
Confirmed: 	23
Tentative: 	30
Rejected: 	56
Iteration: 	15 / 100
Confirmed: 	23
Tentative: 	30
Rejected: 	56
Iteration: 	16 / 100
Confirmed: 	28
Tenta

BorutaPy(alpha=0.05,
         estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                         criterion='mse', max_depth=5,
                                         max_features='auto',
                                         max_leaf_nodes=None, max_samples=None,
                                         min_impurity_decrease=0.0,
                                         min_impurity_split=None,
                                         min_samples_leaf=1,
                                         min_samples_split=2,
                                         min_weight_fraction_leaf=0.0,
                                         n_estimators=203, n_jobs=3,
                                         oob_score=False,
                                         random_state=RandomState(MT19937) at 0x27274C57598,
                                         verbose=0, warm_start=False),
         max_iter=100, n_estimators='auto', perc=100,
         random_state

In [44]:
feat_selector.ranking_

array([ 1,  9,  1,  1,  1,  1,  1,  1, 29, 41, 52, 12, 43,  1, 44, 39, 26,
        1, 59, 46,  2,  1,  1,  1, 54, 31,  3, 61,  1,  1,  1, 12,  5,  6,
       13,  1, 53,  2,  1, 16,  2,  1,  1,  1,  3,  1,  1,  2,  8, 30,  1,
        1,  1,  2,  1,  1,  5,  2,  2, 46, 27, 43,  1,  1, 17, 20, 38, 21,
        1,  2,  1, 23,  1, 16,  2, 22, 49, 24, 35, 26,  7, 40, 18,  1,  2,
       50, 57, 51, 34, 10,  1,  1,  1, 61, 57, 48, 58,  1, 19,  1, 46, 32,
       34, 28, 38,  1, 55, 36, 14])

In [45]:
feat_selector.support_

array([ True, False,  True,  True,  True,  True,  True,  True, False,
       False, False, False, False,  True, False, False, False,  True,
       False, False, False,  True,  True,  True, False, False, False,
       False,  True,  True,  True, False, False, False, False,  True,
       False, False,  True, False, False,  True,  True,  True, False,
        True,  True, False, False, False,  True,  True,  True, False,
        True,  True, False, False, False, False, False, False,  True,
        True, False, False, False, False,  True, False,  True, False,
        True, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
        True,  True,  True, False, False, False, False,  True, False,
        True, False, False, False, False, False,  True, False, False,
       False])

In [50]:
selected = final_features.drop(['ID','WF','Set'],axis=1)[:, feat_selector.support_]

TypeError: '(slice(None, None, None), array([ True, False,  True,  True,  True,  True,  True,  True, False,
       False, False, False, False,  True, False, False, False,  True,
       False, False, False,  True,  True,  True, False, False, False,
       False,  True,  True,  True, False, False, False, False,  True,
       False, False,  True, False, False,  True,  True,  True, False,
        True,  True, False, False, False,  True,  True,  True, False,
        True,  True, False, False, False, False, False, False,  True,
        True, False, False, False, False,  True, False,  True, False,
        True, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
        True,  True,  True, False, False, False, False,  True, False,
        True, False, False, False, False, False,  True, False, False,
       False]))' is an invalid key

In [52]:
final_features.drop(['ID','WF','Set'],axis=1).columns[feat_selector.support_]

Index(['T', 'Wind Speed 100m', 'Wind Direction 100m', 'Wind Speed 10m',
       'Wind Direction 10m', 'T_last_week', 'T_last_month',
       'Wind Direction 100m_last_month', 'Wind Direction 10m_last_month',
       'Wind Speed 100m_Month_Mean', 'Wind Direction 100m_Month_Mean',
       'Wind Speed 10m_Month_Mean', 'Wind Direction 100m_Month_Median',
       'Wind Speed 10m_Month_Median', 'Wind Direction 10m_Month_Median',
       'Wind Speed 10m_Month_Variance', 'T_Distance_Min',
       'Wind Speed 100m_Distance_Max', 'Wind Speed 100m_Distance_Min',
       'Wind Direction 100m_Distance_Max', 'Wind Speed 10m_Distance_Max',
       'Wind Speed 10m_Distance_Min', 'value__sum_values|U_100m',
       'value__fft_coefficient__coeff_0__attr_"real"|U_100m',
       'value__mean|U_100m', 'value__quantile__q_0.7|U_100m',
       'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_10__w_20|U_100m',
       'value__fft_coefficient__coeff_0__attr_"real"|U_10m',
       'value__sum_values|U_10m', 'value__me